In [ ]:
import symd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pickle
import pandas as pd
import skunk
import svglib
import seaborn as sns

In [ ]:
base_colors = ["f94144","f3722c","f8961e","f9844a","f9c74f","90be6d","43aa8b","4d908e","577590","277da1"]    
colors = ['#' + c for c in base_colors]
sns.set_style("white")
sns.set_style("ticks")
sns.set(rc={'axes.facecolor':'#f5f4e9', 
            'grid.color' : '#AAAAAA', 
            'axes.edgecolor':'#333333', 
            'figure.facecolor':'#FFFFFF', 
            'axes.grid': False,
            'axes.prop_cycle':   plt.cycler('color', plt.cm.Dark2.colors),
            'font.family': 'monospace'
           })
print(symd.__version__)

In [ ]:
def run_sim(n, cell, group, w=None, retries=5, pos_frames=0):
    for _ in range(retries):
        try:
            md = symd.Symd(nparticles=n, cell=cell, ndims=3, images=2, force='lj', wyckoffs=w,
              group=group, steps=30000, exeDir='sim', start_temperature=0.5)
            md.remove_overlap()
            md.shrink()
            if pos_frames > 0:
                md.log_positions(frames=pos_frames)
            md.log_output(period = int(1 / md.runParams['time_step']))
            md.run()
            break
        except RuntimeError as e:
            print(e)
            md = None
    return md

In [ ]:
np.random.seed(0)
md = run_sim(5, [8, 8, 8], 17, pos_frames=100)

In [ ]:
plt.plot(md.pe, label='potential')
plt.plot(md.ke, label='kinetic')
plt.plot(md.te, label='total')
plt.legend(loc='best')

## All Sims

In [ ]:
titles = [str(i) for i in range(1,231,14)]
df = pd.DataFrame()
retries = 3
def smooth(x):
    window = int(np.ceil(len(x) / 10.0))
    weights = np.ones(window)
    energy_smooth = np.convolve(
        weights / weights.sum(), x[(window + 1):], mode="valid"
    )
    return x

def standardize(te):
    i = int(md.te.shape[0] * 0.2)
    te = md.te[i:]
    return te - np.mean(te)

In [ ]:
for i,t in enumerate(titles):    
    md = run_sim(5, [8, 8, 8], i+1)
    df = df.assign(**{t: standardize(md.te)})    

In [ ]:
fig = plt.figure(figsize=(4,5.7))
ax = plt.gca()
mx = df.shape[0] // 2
for i,n in enumerate(df.columns):
    color = colors[i % len(base_colors)]
    ax.plot(df[n] + i, color=color)
    offsetbox = mpl.offsetbox.TextArea(n)
    #mx = (df.shape[0] - 1) * (i / df.shape[1])
    #if i % 2 == 0:
    #    mx = df.shape[0] - mx - 1
    ab = mpl.offsetbox.AnnotationBbox(offsetbox, (mx,i),
                    xybox=(mx,i),
                    xycoords='data',
                    boxcoords='data',
                    arrowprops=None,
                    bboxprops=dict(fc="#f5f4e9", lw=0))
    ax.add_artist(ab)
ax.set_xlabel(r'Time [$\tau$]')
ax.set_facecolor('#f5f4e9')
ax.set_ylabel(r'$\Delta$ Energy [$\epsilon$]')
plt.savefig('energy3d.svg')

## Atlas

In [ ]:
def rmsd(p1, p2):
    return np.mean((p1 - p2)**2, axis=(1,2))

In [ ]:
def crystal(n, cell, group, w=None, retries=2, steps=10**6, ndims=3):
    # adjust for group size
    groupd = symd.groups.load_group(group, ndims)
    m = len(groupd['genpos'])
    n = max(2, n // m)
    total = n * m
    if w is not None:        
        n += sum(w)
        for l in range(len(w)):
            total += w[l] * groupd['specpos'][l]['size']
        name = f'{group}-{n}-{sum(w)}'
    else:
        name = f'{group}-{n}'
    print('Simulating', n, 'particles:',name, 'cell will have', total)    
    # break out the try/except because we will accept failed NPT (because it jams so hard)
    for i in range(retries):
        # NPT
        md = symd.Symd(nparticles=n, cell=cell, ndims=ndims, images=2, force='lj', wyckoffs=w,
          group=group, steps=steps, exeDir=f'crystal-{name}', pressure=0.5, temperature=0.1, start_temperature=0.2)
        try:        
            md.remove_overlap()
            md.shrink()
        except RuntimeError as e:
            continue
        md.log_positions()
        try:
            md.run()
        except RuntimeError as e:
            pass

        # NVT
        md.runParams['temperature'] = 0.05
        md.runParams['Pressure'] = None
        md.runParams['box_update_period'] = 0
        md.runParams['steps'] = steps // 10
        md.log_positions(filename='equil.xyz')
        try:
            md.run()
        except RuntimeError as e:
            continue
        config = md.positions[-1]

        # Stability
        fp = np.loadtxt(md.runParams['final_positions'])
        # changing group, so need to read normed cell
        cell = md.read_cell(normed=True)
        m = fp.shape[0]
        md2 = symd.Symd(nparticles=m, cell=cell, ndims=ndims, images=2, force='lj', wyckoffs=None,
          group=1, steps=5000, exeDir=f'melt-{name}', temperature=None, start_temperature=0)
        # run once to get melting traj
        # then again for longer with longer period
        md2.log_positions(period=5)
        md2.runParams['start_positions'] = md.runParams['final_positions']            
        try:
            md2.run()
        except RuntimeError as e:
            continue
        traj = md2.positions
        md2.runParams['steps'] = steps // 10
        md2.log_positions(period=int(1 / md2.runParams['time_step']))
        #md2.run()
        csm = rmsd(md2.positions[:,:m], md2.positions[0,:m])
        #csm = []
        #for i in range(md2.positions.shape[0]):
        #    csm.append(compute_symm(md2.positions[i], group, md2.read_cell(), ndims, n))
        return config, md2.positions[-1], csm, traj, np.arange(0,5000,5) * md2.runParams['time_step']
    return None

In [ ]:
def plot_config(pos, figsize=(5,5), color='#333333'):
    N, D = pos.shape    
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(projection='3d')        
    
    points = ax.plot(pos[:,0], pos[:,1], pos[:,2], color=color, marker='o', linestyle='None', mec='#999')
    ax.set_facecolor('#f5f4e9')
    fig.patch.set_facecolor('#f5f4e9')
    ax.axis('off')    
    
    plt.tight_layout()
    #return skunk.pltsvg(fig=fig)
    return points

In [ ]:
config, config2, csm, traj, time = crystal(96, [25, 25, 25], 180, w=[1] * 4)

In [ ]:
p = plot_config(traj[0])

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(config[:,0], config[:,1], '.')
plt.plot(config2[:,0], config2[:,1], '.')

In [ ]:
plt.plot(time, csm)

In [ ]:
are you sure you want to run this?

from multiprocessing import Pool

cdf = None
results = []
trajs = {}

with Pool(8) as pool:
    for N in [8,16,32,128]:
        cell = [25, 25]
        if N == 128:
            cell = [35, 35]
        for i,t in enumerate(titles):
            W = len(symd.groups.load_group(i+1, 2)['specpos'])    
            for j in range(1 + W):
                wycks = None if j == 0 else [1] * j
                name = f'{t}-w{j}-n{N}'
                job = pool.apply_async(crystal, (N, cell, i+1, wycks))
                #job = crystal(N, [25, 25], i+1, wycks)
                results.append((t, name, N, j, job))
                
    for r in results:        
        t, name, N, j, ar = r
        print('Getting result for ', name)
        res = ar.get()
        #res = ar
        if res is None:
            continue
        config, config2, csm, traj, time = res
        T = len(csm)
        
        df2 = pd.DataFrame({'Group':[t] * T, 'Traj': [name] * T, 
                            'N':[N] * T, 'Wyckoffs':[str(j)] * T, 'RMSD':csm, 'Time':time})
        if cdf is None:
            cdf = df2
        else:
            cdf = pd.concat((cdf, df2))
        trajs[name] = traj
        
cdf.reset_index(inplace=True)
cdf.to_pickle('atlas2d.pkl.gz')
with open('atlas2d.traj.pkl', 'wb') as f:
    pickle.dump(trajs, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
cdf = pd.read_pickle('atlas2d.pkl.gz')
with open('atlas2d.traj.pkl', 'rb') as f:
    trajs = pickle.load(f)

In [ ]:
replaces = []
def annotate_config(rmsd, traj, time, wyckoffs, color):
    late_rmsd = rmsd.where(time > 15)
    idx = late_rmsd.argmin()
    label = traj.iloc[idx]
    w = wyckoffs.iloc[idx]
    replaces.append((label, f'C{w}'))
    x = time.iloc[idx]
    y = rmsd.iloc[idx]
    ax = plt.gca()
    box = skunk.Box(25, 25, label)    
    ab = mpl.offsetbox.AnnotationBbox(box, (x, y),
                        pad=0,
                        bboxprops=dict(edgecolor='#333', linewidth=1),
                        xybox=(0.2, 0.7),
                        xycoords='data',
                        boxcoords='axes fraction',
                        arrowprops=dict(arrowstyle="->",color='#333'))

    ax.add_artist(ab)

g = sns.relplot(data=cdf, x='Time', y='RMSD', 
        kind='line', hue='Wyckoffs', col='Group', col_wrap=6,
        aspect=1, linewidth=1, style='N', height=1.75, palette='Dark2',
               hue_order=[str(i) for i in range(9)])
plt.ylim(0,1)
sns.move_legend(g, "lower right", bbox_to_anchor=(0.9,0.1), ncol=3)
g.map(annotate_config, 'RMSD', 'Traj', 'Time', 'Wyckoffs').set_axis_labels("Time", "RMSD")


main_svg = skunk.pltsvg()
svg = skunk.insert({l: plot_config(trajs[l][0], color=c) for l,c in replaces}, svg=main_svg)
skunk.display(svg)
with open('atlas.svg', 'w') as f:
    f.write(svg)

## Movie

In [ ]:
# make a movie
import moviepy.editor as editor
from moviepy.video.io.bindings import mplfig_to_npimage

def plot_traj(traj, M, title='@_172135352171_', color='#333333', fps=60):
    T, N, D = traj.shape        
    fps = fps
    duration = T / fps + 1
    dpi = 90
    fig = plt.figure(figsize=(1200 / dpi, 800 / dpi), dpi=dpi)
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    tc = [np.random.choice(colors) for _ in range(N)]
    points1 = [ax1.scatter(traj[0,:,0], traj[0,:,1], traj[0,:,2], 
                          color=tc,
                          marker='o')]
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    points2 = ax2.plot(traj[0,:M,0], traj[0,:M,1], traj[0,:M,2], c=color, mec='#999', marker='o',                        
                        linestyle='None', markersize=18)[0]
    ax2.set_facecolor('#f5f4e9')
    ax1.set_facecolor('#f5f4e9')
    fig.patch.set_facecolor('#f5f4e9')
    title = ax.set_title(title, fontsize=32, color='#333333',fontname='monospace')
    ax1.axis('off')
    ax2.axis('off')
    angle = 60
    plt.tight_layout()
    def make_frame(t):
        i = int((t - 1)  * fps)
        i = max(0, min(i, T-1))
        points1[0].remove()
        points1[0] = ax1.scatter(traj[i,:,0], traj[i,:,1], traj[i,:,2], 
                          color=tc,
                          marker='o')
        points2.set_data_3d(traj[i,:M,0], traj[i,:M,1], traj[i,:M,2])    
        ax1.view_init(30, (angle + t * 3) % 360)
        ax2.view_init(30, (angle + t * 3) % 360)
        plt.draw()
        return mplfig_to_npimage(fig)

    return editor.VideoClip(make_frame, duration=duration)
def write_video(clips, output, fps=60, transition=0.25):
    composite = editor.concatenate(clips[:1] + [c.crossfadein(transition) for c in clips[1:]], 
                                   padding=-transition, method='compose')   
    composite.write_videofile(output, fps=fps, preset='slower', ffmpeg_params=['-tune', 'animation'])

In [ ]:
c = plot_traj(traj, 120, title=t)
write_video([c], '3d.mp4')

In [ ]:
k = 25
top = cdf[cdf.Time == cdf.Time.max()].sort_values(by=['RMSD']).Traj.values[:k]

clips = []
for t in top:
    c = plot_traj(trajs[t][:60 * 3], title=t)
    clips.append(c)
write_video(clips, 'top.mp4')

In [ ]:
k = 25
top = cdf[cdf.Time == cdf.Time.max()].sort_values(by=['RMSD']).Traj.values[-k:]

clips = []
for t in top:
    c = plot(trajs[t][:60 * 3], title=t)
    clips.append(c)
write_video(clips, 'bottom.mp4')